In [1]:
#|default_exp init

# Initialization

In [8]:
#|export
import pickle
import gzip
import math
import sys
import gc
import os
import time
import traceback
import shutil
import torch
import matplotlib.pyplot as plt
import numpy as np
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter, itemgetter
from functools import partial
from copy import copy

from torch import nn
import torchvision.transforms.functional as TF
from torch import nn, tensor, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, default_collate
from torch.nn import init
from torcheval.metrics import MulticlassAccuracy
from datasets import load_dataset, load_dataset_builder

from miniai.datasets import * 
from miniai.conv import * 
from miniai.learner import * 
from miniai.activations import * 

In [9]:
from fastcore.test import test_close
plt.style.use('fast')
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)

import logging; logging.disable(logging.WARNING)
set_seed(42)

In [10]:
xl, yl = 'image', 'label'
name = "fashion_mnist"
dsd = load_dataset(name)

@inplace
def transformi(b): b[xl] = [TF.to_tensor(o) for o in b[xl]]

  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
bs = 1024
tds = dsd.with_transform(transformi)
dls = DataLoaders.from_dd(tds, bs, num_workers=4)

In [12]:
dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

(torch.Size([1024, 1, 28, 28]), tensor([5, 7, 4, 7, 3, 8, 9, 5, 3, 1]))

In [14]:
def get_model():
    return nn.Sequential(
        conv(1, 8), conv(8, 16), conv(16, 32), 
        conv(32, 64), conv(64,10, act=False)).to(device)

In [ ]:
learn = MomentumLearner(get_model(), dls, F.cross_entropy, cbs=[DeviceCB()])